In [ ]:
# default_exp templates

# templates

> This module defines `Templates` for embedding arbitrary iterable and replaceable expressions into blocks of text, allowing for metaprogramming and automation of tasks in other languages through python, or simply filling out forms.

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import warnings
with warnings.catch_warnings(): #ignore warnings
    warnings.simplefilter("ignore")
    from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable
    import numpy as np
    from sidis.conversion import cast
    from sidis.recursion import get

In [ ]:
#export
def replace(l,**kwargs):
    'Replace the line `l` in `txt` with any occurances of the dictionaried values. '
    for k,v in kwargs.items():
        l=l.replace(k,str(v))
    return l

def txt2lst(txt):
    'Remove all newlines `\n` and empty strings `''` from the template `txt`.'
    return [t for t in txt.split('\n') if t!='']

def lst2txt(ltxt):
    'Concatenate a list of text `ltxt` into a string separated by newlines.'
    return '\n'.join(ltxt)

def ZIP(txt,_iter,*lambdas,as_txt=False):
    '''Loop over the iterable `_iter` and format `txt` in order of the `lambdas`.
    If `_iter` is an int, it is treated as `range(_iter)`, and if it is a tuple, 
    it is treated as `np.ndindex(_iter)`.
    '''
    _iter=cast(_iter,list)
    try:
        l=[str(txt).format( *[l(*i) for l in lambdas] ) for i in _iter]
    except:
        l=[str(txt).format( *[l(i) for l in lambdas] ) for i in _iter]
    if as_txt is True:
        l=lst2txt(l)
    return l

def getEvals(replaced_txt,funcs=['ZIP']):
    '''Obtain a tuple of containing the `funcs`, the text they format, their arguments, 
    and their line index in the template `txt`.
    '''
    y=[]
    for i,o in enumerate(replaced_txt):
        for f in funcs:
            if len(o.split(f))>1:
                #function, (txt to be formatted, func args), position in file
                y+=[(f,o.split(f),i)]
    return y

def filltxt(txt,funcs=['ZIP'],**kwargs):
    '''Take a template `txt`, replace all `kwargs` via `Replace`, then evaluate the `funcs` 
       on the surrounding text using `GetEvals`.'''
    txt=txt2lst(txt)
    for i,l in enumerate(txt):
        txt[i]=replace(l,**kwargs)
    y=getEvals(txt,funcs)
    e=list(map(lambda s: eval(s[0]+'('+'{0:s[1][0]}[0]'+','+s[1][1]+')' ),y)) #evaluate the function 
    u=[(i,e) for i,e in zip([y[i][-1] for i in range(len(y))],[e[i] for i in range(len(y))])] #position/evals
    t=[] #new txt
    for i in range(len(txt)):
        t+=[txt[i]]
        for j,e in u: #remove old lines
            if i==j:
                t.remove(txt[i])
                for q in e:
                    t+=[q]
    return t

        
class Template:
    '''
    Automates iteration over arbitrary Python functions embedded into blocks of text.
    
    Class attrs:
        
        funcs=[`ZIP`] (list of callable) : the keyword function to map over the text it's embedded in
        
        filler=`fill` (callable) : method of filling the text
        
    
    Inputs:
    
        `temp` (str): input string to be filled, or file name to be loaded
        
        `kwargs` (dict): characters to be replaced and evaluated
        
        
    Attrs:
    
        `plate` (list): line-by-line filling of `temp` based on `kwargs`
        
        `temp` (str): saved version of `temp` separated by newlines
        
        
    Methods:
    
        `load`: loads `fname` and fills based on `kwargs`.
        
        `out`: write/append `txt`, which defaults to filled data `str`, to `fname`.
        
        `fill`: fills the template by replacing `kwargs` and evaluating `funcs`.
        
    '''
    funcs=['ZIP']
    filler=filltxt
    def __init__(self,temp,**kwargs):
        self.__dict__.update({k:v for k,v in kwargs.items() if k!='self' and k!='kwargs'})
        self.temp=temp
        try:
            self.load(temp,**kwargs)
            print(f'Loaded {temp}.')
        except:
            self.temp=temp
            self.fill(**kwargs)

    def fill(self,temp=None,append=False,**kwargs):
        if not append:
            if temp is not None:
                self.temp=temp
            if kwargs!={}:
                self.plate=Template.filler(self.temp,Template.funcs,**kwargs)
                self.__dict__.update(kwargs)
        else:
            if (temp is not None) and (temp!=self.temp):
                self.temp+='\n'+temp
            if kwargs!={}:
                self.plate+=Template.filler(temp,Template.funcs,**kwargs)
                self.__dict__.update(kwargs)
            
    def txt(self):
        return lst2txt(self.plate)
            
    def load(self,fname,**kwargs):
        with open(fname,'r') as f:
            self.temp=f.read()
        self.fill(**kwargs)
            
    def out(self,fname,txt=None,s='a'):
        if txt is None:
            txt=self.txt()
        with open(fname,s) as f:
            f.write(txt)
        
    def __repr__(self):
        try:
            return self.txt()
        except:
            return self.temp       

In [ ]:
show_doc(Template)

<h2 id="Template" class="doc_header"><code>class</code> <code>Template</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Template</code>(**`temp`**, **\*\*`kwargs`**)

Automates iteration over arbitrary Python functions embedded into blocks of text.

Class attrs:
    
    funcs=[[`ZIP`](/sidis/templates.html#ZIP)] (list of callable) : the keyword function to map over the text it's embedded in
    
    filler=[`fill`](/sidis/conversion.html#fill) (callable) : method of filling the text
    

Inputs:

    `temp` (str): input string to be filled, or file name to be loaded
    
    `kwargs` (dict): characters to be replaced and evaluated
    
    
Attrs:

    `plate` (list): line-by-line filling of `temp` based on `kwargs`
    
    `temp` (str): saved version of `temp` separated by newlines
    
    
Methods:

    [`load`](/sidis/utils.html#load): loads `fname` and fills based on `kwargs`.
    
    `out`: write/append `txt`, which defaults to filled data `str`, to `fname`.
    
    [`fill`](/sidis/conversion.html#fill): fills the template by replacing `kwargs` and evaluating `funcs`.
    

In [ ]:
t=Template('''These characters get replaced: _variable, _function

This line is then formatted: {0} ZIP _variable , _function

And this one is iterated: {0} ZIP _iter , _function''')

In [ ]:
t.fill(_variable=2,_function="lambda i:i*10",_iter=range(2))
t

These characters get replaced: 2, lambda i:i*10
This line is then formatted: 20 
And this one is iterated: 0 
And this one is iterated: 10 

What happened? When calling `t.fill`, `_variable` was replaced with `2` everywhere it appeared in the template, similarly with `_function`. This includes after the custom function `ZIP`, where they appear as arguments. 

After this substitution, `ZIP` formatted the text preceding it (`{0}`) with the result of `_function(_variable)`, i.e, `format("{0}",(lambda i:i*10)(2))`. 

In the next line, `ZIP` repeatedly formatted the text preceding it (`{0}`) with the result of `_function(_iter)`, i.e, `'\n'.join([format("{0}",(lambda i:i*10)(j)) for j in range(2)])`. 

Before we go into more detail on these functions, let's examine the template object:

In [ ]:
t.__dict__

{'temp': 'These characters get replaced: _variable, _function\n\nThis line is then formatted: {0} ZIP _variable , _function\n\nAnd this one is iterated: {0} ZIP _iter , _function',
 'plate': ['These characters get replaced: 2, lambda i:i*10',
  'This line is then formatted: 20 ',
  'And this one is iterated: 0 ',
  'And this one is iterated: 10 '],
 '_variable': 2,
 '_function': 'lambda i:i*10',
 '_iter': range(0, 2)}

The `temp` attribute stores the un-formatted text handed to `Template`. The `plate` attribute stores the formatted lines as a list of strings. The object also stores any kwargs as attrs. Finally, `text()` attribute concatenates the formatted text into a string separated by newlines, and is used as the `__repr__`. 

Let's see how `ZIP` works:

In [ ]:
show_doc(ZIP)

<h4 id="ZIP" class="doc_header"><code>ZIP</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

> <code>ZIP</code>(**`txt`**, **`_iter`**, **\*`lambdas`**, **`as_txt`**=*`False`*)

Loop over the iterable `_iter` and format `txt` in order of the `lambdas`.
If `_iter` is an int, it is treated as `range(_iter)`, and if it is a tuple, 
it is treated as `np.ndindex(_iter)`.

In [ ]:
ZIP('{0}',range(10),lambda i:i)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

When embedded in the `Template`, `ZIP` treats the text proceeding it as an iterable or a single-substitution, and any text after the `,` as a list of functions. These functions take the iterable as an argument, and format the text preceding `ZIP` with their evaluation on the iterable. They do not need to be lambdas - any callable will do. This makes embedding `ZIP` into other text documents a useful way to modularize the formatting of text, and automate code blocks in other languages which lack Python flexibility. 

Other helper functions enabling this process are as follows:

In [ ]:
show_doc(replace)

<h4 id="replace" class="doc_header"><code>replace</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>replace</code>(**`l`**, **\*\*`kwargs`**)

Replace the line `l` in `txt` with any occurances of the dictionaried values. 

In [ ]:
replace('''result={0}; ZIP _myvar,lambda i:i*10''',_myvar=2)

'result={0}; ZIP 2,lambda i:i*10'

In [ ]:
show_doc(txt2lst)

<h4 id="txt2lst" class="doc_header"><code>txt2lst</code><a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h4>

> <code>txt2lst</code>(**`txt`**)

Remove all newlines `
` and empty strings `` from the template `txt`.

In [ ]:
txt=txt2lst('''var=_myvar
result={0}; ZIP _myvar,lambda i:i*10
''')
txt

['var=_myvar', 'result={0}; ZIP _myvar,lambda i:i*10']

In [ ]:
show_doc(getEvals)

<h4 id="getEvals" class="doc_header"><code>getEvals</code><a href="__main__.py#L30" class="source_link" style="float:right">[source]</a></h4>

> <code>getEvals</code>(**`replaced_txt`**, **`funcs`**=*`['ZIP']`*)

Obtain a tuple of containing the `funcs`, the text they format, their arguments, 
and their line index in the template `txt`.

In [ ]:
for i,l in enumerate(txt):
    txt[i]=replace(l,_myvar=2)
getEvals(txt,funcs=['ZIP'])

[('ZIP', ['result={0}; ', ' 2,lambda i:i*10'], 1)]

Text blocks of this kind (`ZIP`, follows by text to format, follows by arguments after comma) are extracted using `filltxt`:

In [ ]:
show_doc(filltxt)

<h4 id="filltxt" class="doc_header"><code>filltxt</code><a href="__main__.py#L42" class="source_link" style="float:right">[source]</a></h4>

> <code>filltxt</code>(**`txt`**, **`funcs`**=*`['ZIP']`*, **\*\*`kwargs`**)

Take a template `txt`, replace all `kwargs` via `Replace`, then evaluate the `funcs` 
on the surrounding text using `GetEvals`.

These are then packaged into the `Template` class, which we give another arbitrary example of here:

In [ ]:
t=Template('''Filling out _stuff

which {0}{1} is ZIP 1, lambda t:'iterates ' , _arbitrarily

{0} ZIP range(3), lambda t: "much"

easier

since we can embed any keyword, iterable, and function anywhere.

''',_stuff='stuff',_arbitrarily="lambda t: 'arbitrarily'")
t

Filling out stuff
which iterates arbitrarily is 
much 
much 
much 
easier
since we can embed any keyword, iterable, and function anywhere.

In [ ]:
t.fill('we can append, _too',_too='too!',append=True)

In [ ]:
t

Filling out stuff
which iterates arbitrarily is 
much 
much 
much 
easier
since we can embed any keyword, iterable, and function anywhere.
we can append, too!

Finally, a `Template` will attempt to auto-load a file if the input is a filename, e.g `Template('template.txt')` will load the file `template.txt` if it exists in the current directory, and use that as the string to format.

In [ ]:
#hide
#notebook2script()